In [31]:
import pandas as pd
import os

In [32]:
fecha_inicio="2022/09/01"
fecha_fin="2022/09/30"

mes_referencia=fecha_inicio[5:7]
ano_referencia=fecha_inicio[2:4]


In [33]:
def suma_mes(mes):
    mes_cadena=str(int(mes)+1)
    if len(mes_cadena)==1:
        mes_cadena="0"+ mes_cadena
    return mes_cadena

In [34]:
tramas=[]
for (root,dirs,files) in os.walk('TRAMAS'):
    for name in files:
        if name[0:1]=="M":
            dia=name[12:14]
            mes=name[15:17]
            ano=name[20:22]
            if (mes_referencia==mes and ano_referencia == ano) or (dia=="01" and suma_mes(mes_referencia)==mes):
                tramas.append(root + "\\"+ os.path.join(name))

In [35]:
# pandas concat csv files
df = pd.concat((pd.read_csv(f, sep=";", encoding="latin-1", header=None, skiprows=1) for f in tramas), ignore_index=False)
df.columns =  ["Tipo", "Estacion", "Fecha", "Hora_inicio", "Mezcla", "Premezcla", "Batch", "Receta", "Kg_corcho", "Minutos_pesada", "Kg_DU", "Tiempo_vaciado", "Tiempo_inyeccion_agua", "Kg_Agua", "Tiempo_mezcla_agua", "Tiempo_inyeccion_cola", "Kg_cola", "Tiempo_mezcla_cola", "Tiempo_vaciado_mezcla", "Molde_inicial_ext1", "Molde_final_ext1", "Receta", "Molde_inicial_ext2", "Molde_final_ext2", "Tapones_buenos_g1", "Tapones_malos_g1", "Tiempo_extraccion_g1", "Tapones_buenos_g2", "Tapones_malos_g2", "Tiempo_extraccion_g2", "Tapones_buenos_descartados_g1", "Tapones_buenos_descartados_g2", "nada_1", "nada_2"]

In [36]:
df.drop(columns=["Tipo", "Hora_inicio", "Premezcla", "Batch", "Receta", "Minutos_pesada", "Tiempo_vaciado", "Tiempo_inyeccion_agua", "Kg_Agua", "Tiempo_mezcla_agua", "Tiempo_inyeccion_cola", "Tiempo_mezcla_cola", "Tiempo_vaciado_mezcla", "Molde_inicial_ext1", "Molde_final_ext1", "Receta", "Molde_inicial_ext2", "Molde_final_ext2", "Tiempo_extraccion_g1", "Tiempo_extraccion_g2","nada_1", "nada_2"], inplace=True)	

In [37]:
df=df.loc[:, ["Fecha", "Estacion", "Mezcla", "Kg_corcho", "Kg_cola", "Kg_DU", "Tapones_buenos_g1", "Tapones_buenos_g2", "Tapones_malos_g1", "Tapones_malos_g2", "Tapones_buenos_descartados_g1", "Tapones_buenos_descartados_g2"]]

In [38]:
df.reset_index(level=0, inplace=True)
df.drop(columns=["index"], inplace=True)

In [39]:
tf=(df["Tapones_malos_g1"]==65492)
error_cantidad_malos= round((len(tf[tf==True])/len(df))*100,2)
error_cantidad_malos

7.71

In [40]:
df.loc[tf, "Tapones_malos_g1"]=0
df2 = df.copy() 

In [41]:
df2.loc[:, "Kg_corcho"]=df2["Kg_corcho"]/1000
df2.loc[:, "Kg_cola"]=df2["Kg_cola"]/1000
df2.loc[:, "Kg_DU"]=df2["Kg_DU"]/1000
df2.loc[:, "Tapones_buenos"]=(df2["Tapones_buenos_g1"]+df2["Tapones_buenos_g2"])/1000
df2.loc[:, "Tapones_malos"]=(df2["Tapones_malos_g1"]+df2["Tapones_malos_g2"])/1000
df2.loc[:, "Tapones_buenos_descartados"]=(df2["Tapones_buenos_descartados_g1"]+df2["Tapones_buenos_descartados_g2"])/1000
df2.drop(columns=["Tapones_buenos_g1", "Tapones_buenos_g2", "Tapones_malos_g1", "Tapones_malos_g2", "Tapones_buenos_descartados_g1", "Tapones_buenos_descartados_g2"], inplace=True)
df2["Estacion"].replace([10], "MA10", inplace=True)
df2["Estacion"].replace([1], "MA11", inplace=True)
df2["Estacion"].replace([12], "MA12", inplace=True)
df2["Estacion"].replace([13], "MA13", inplace=True)
df2["Estacion"].replace([14], "MA14", inplace=True)
df2["Estacion"].replace([15], "MA15", inplace=True)

In [42]:
df2. drop_duplicates(["Fecha", "Estacion", "Mezcla"], keep="first", inplace=True)

In [43]:
df2["Fecha"]=pd.to_datetime(df2["Fecha"], format="%d/%m/%y")
df2.sort_values(by=["Fecha", "Estacion", "Mezcla"], inplace=True)

df2=df2.loc[(df2["Fecha"]>=fecha_inicio) & (df2["Fecha"]<=fecha_fin)]

In [44]:
error_mezclas_duplicadas=round((1-(len(df2)/len(df)))*100,2)
error_mezclas_duplicadas

28.16

In [45]:
#create csv file with commain float fields
df2.to_csv("tramas_generado.csv", sep=";", encoding="latin-1", index=False,decimal=",")